In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import tensorflow
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [6]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_08d7a93587f443f9a626fcf7aa8e4139 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='OrHVdRqlCCJ7t-zuEJeHrRUDRpaaquqDASL00_V4WFjI',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

streaming_body_1 = client_08d7a93587f443f9a626fcf7aa8e4139.get_object(Bucket='icebergdetectioninsatelliteimages-donotdelete-pr-fd92jemwiaber5', Key='dataset.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [7]:
from io import BytesIO
import zipfile
unzip =zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [8]:
pwd


'/home/wsuser/work'

In [9]:
x_train = train_datagen.flow_from_directory(r"/home/wsuser/work/dataset/train",target_size=(64,64),batch_size= 32, class_mode='binary')
x_test = test_datagen.flow_from_directory(r"/home/wsuser/work/dataset/test",target_size=(64,64),batch_size= 32, class_mode='binary')

Found 1284 images belonging to 2 classes.
Found 320 images belonging to 2 classes.


In [12]:
model =Sequential()

# Convolutional layer and maxpool layer 1
model.add(Conv2D(512,(3,3),activation='relu',input_shape=(64,64,3)))
model.add(MaxPooling2D(2,2))

# Convolutional layer and maxpool layer 2
model.add(Conv2D(512,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))

# Convolutional layer and maxpool layer 3
model.add(Conv2D(256,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))

# Convolutional layer and maxpool layer 4
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))

# This layer flattens the resulting image array to 1D array
model.add(Flatten())

# Hidden layer with 512 neurons and Rectified Linear Unit activation function 
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.5))
# Output layer with single neuron which gives 0 for Cat or 1 for Dog 
#Here we use sigmoid activation function which makes our model output to lie between 0 and 1
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 62, 62, 512)       14336     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 31, 31, 512)      0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 29, 29, 512)       2359808   
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 14, 14, 512)      0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 12, 12, 256)       1179904   
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 6, 6, 256)       

In [14]:
model.fit(x_train,  epochs=10, validation_data=x_test)

Epoch 1/10
41/41 [==============================] - 199s 5s/step - loss: 0.6997 - accuracy: 0.5117 - val_loss: 0.6936 - val_accuracy: 0.5000
Epoch 2/10
41/41 [==============================] - 195s 5s/step - loss: 0.6878 - accuracy: 0.5374 - val_loss: 0.6442 - val_accuracy: 0.6438
Epoch 3/10
41/41 [==============================] - 196s 5s/step - loss: 0.6572 - accuracy: 0.5561 - val_loss: 0.6051 - val_accuracy: 0.7156
Epoch 4/10
41/41 [==============================] - 196s 5s/step - loss: 0.5762 - accuracy: 0.6893 - val_loss: 0.5247 - val_accuracy: 0.7406
Epoch 5/10
41/41 [==============================] - 196s 5s/step - loss: 0.5279 - accuracy: 0.7204 - val_loss: 0.4893 - val_accuracy: 0.7594
Epoch 6/10
41/41 [==============================] - 198s 5s/step - loss: 0.5046 - accuracy: 0.7391 - val_loss: 0.4713 - val_accuracy: 0.7594
Epoch 7/10
41/41 [==============================] - 195s 5s/step - loss: 0.4745 - accuracy: 0.7547 - val_loss: 0.4236 - val_accuracy: 0.7812
Epoch 8/10
41

In [15]:
model.save("iceberg.h5")

In [16]:
!tar -zcvf iceberg-detection-model.tgz iceberg.h5

iceberg.h5


In [17]:
ls -1

dataset/
iceberg-detection-model.tgz
iceberg.h5


In [18]:
%pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 23.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [19]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
     "url": "https://us-south.ml.cloud.ibm.com",
    "apikey":"vmGKwS7E4QkrTod9nQw7OEgiNNx-wq0I5AfN4hrD6PYG"
}
client=APIClient(wml_credentials)

In [21]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name)['metadata']['id'])

In [22]:
space_uid=guid_from_space_name(client,'models')
print("Space UID = "+space_uid)

Space UID = 024cde9e-3350-4d1d-8f64-2e7e7a651999


In [23]:
client.set.default_space(space_uid)

'SUCCESS'

In [24]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1-py3.9             12b83a1

In [26]:
software_spec_uid = client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [27]:
model_details = client.repository.store_model(model='iceberg-detection-model.tgz',meta_props={
client.repository.ModelMetaNames.NAME:"CNN",
client.repository.ModelMetaNames.TYPE:"keras_2.2.4",
client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid}
                                              )
model_id = client.repository.get_model_uid(model_details)
    

Note: Warnings!! :  Model type keras_2.2.4 is deprecated. We recommend you use a supported model type. See Supported Frameworks https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/pm_service_supported_frameworks.html
This method is deprecated, please use get_model_id()


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1452: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [28]:
model_id

'af56ebde-5679-4252-8ce2-227ff82d9408'

In [29]:
client.repository.download(model_id, 'my_model.tar.gz')

Successfully saved model content to file: 'my_model.tar.gz'


'/home/wsuser/work/my_model.tar.gz'

In [30]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [31]:
model = load_model("iceberg.h5")